# Trainingspipeline 10

- ohne Bigramme
- vector_size: 400
- window: 3
- min_count: 2
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [4]:
df.shape

(304129, 10)

In [5]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [6]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [7]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

## Training von Word2Vec

In [8]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [9]:
# Trainingsparamter setzen

vector_size = 400                  # Dimensionality of the word vectors
window = 3                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

### Zeitraum 1: 1700-1750

In [10]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [11]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 18 s


In [12]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [13]:
len(sentences1)

113328

In [14]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 23s


In [15]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('desolazione', 0.8122466802597046),
 ('macello', 0.8015394806861877),
 ('rumore', 0.7973864078521729),
 ('spavento', 0.7878577709197998),
 ('sacco', 0.7869684100151062),
 ('precipitoso', 0.782479465007782),
 ('sconfiggere', 0.771727442741394),
 ('naviglio', 0.7699798941612244),
 ('agitazione', 0.7696977257728577),
 ('romore', 0.7684490084648132)]

In [16]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2Vec10', '10w2v1.model'))

### Zeitraum 2: 1751-1800

In [17]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [18]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 17.9 s


In [19]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 18s


In [20]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('tumulto', 0.8902384042739868),
 ('scherno', 0.8897624015808105),
 ('disperazione', 0.8894420862197876),
 ('oblio', 0.8871777057647705),
 ('spavento', 0.8866683840751648),
 ('servaggio', 0.8830365538597107),
 ('letargo', 0.8828788995742798),
 ('usbergo', 0.8797255158424377),
 ('varco', 0.8792508244514465),
 ('usurpatore', 0.8763269186019897)]

In [21]:
w2v2.save(os.path.join('trained_models/Word2Vec10', '10w2v2.model'))

### Zeitraum 3: 1801-1825

In [22]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [23]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 15.6 s


In [24]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 21s


In [25]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('infiammare', 0.8956483006477356),
 ('forsennato', 0.8915051221847534),
 ('avversa', 0.8911566734313965),
 ('turpe', 0.8786180019378662),
 ('furore', 0.8784270882606506),
 ('rissa', 0.8744924068450928),
 ('impotente', 0.8704537153244019),
 ('orrore', 0.8686704635620117),
 ('inazione', 0.8647351264953613),
 ('opprimere', 0.861552894115448)]

In [26]:
w2v3.save(os.path.join('trained_models/Word2Vec10', '10w2v3.model'))

### Zeitraum 4: 1826-1850

In [27]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [28]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 15.7 s


In [29]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 18s


In [30]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('furore', 0.8684925436973572),
 ('opprimere', 0.8399512767791748),
 ('spavento', 0.8213335275650024),
 ('odio', 0.8177584409713745),
 ('servaggio', 0.8155475854873657),
 ('sdegno', 0.8127018809318542),
 ('smisurato', 0.8095069527626038),
 ('cagionare', 0.8049303293228149),
 ('impeto', 0.8042721152305603),
 ('agitazione', 0.80405592918396)]

In [31]:
w2v4.save(os.path.join('trained_models/Word2Vec10', '10w2v4.model'))

### Zeitraum 5: 1851-1875

In [32]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [33]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.6 s


In [34]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 15s


In [35]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('disperazione', 0.8839731216430664),
 ('affannoso', 0.8824787735939026),
 ('fremito', 0.880957841873169),
 ('orrore', 0.8740189075469971),
 ('sgomento', 0.8659651279449463),
 ('esaltazione', 0.8592141270637512),
 ('spavento', 0.8567482829093933),
 ('abbattimento', 0.8549591302871704),
 ('ribrezzo', 0.8543979525566101),
 ('brivido', 0.8539121150970459)]

In [36]:
w2v5.save(os.path.join('trained_models/Word2Vec10', '10w2v5.model'))

### Zeitraum 6: 1876-1900

In [37]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [38]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 18.1 s


In [39]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 13s


In [40]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.8050877451896667),
 ('violento', 0.8015614151954651),
 ('orrore', 0.7975033521652222),
 ('sconforto', 0.791452944278717),
 ('furore', 0.7914022207260132),
 ('disperazione', 0.7911666631698608),
 ('indicibile', 0.787901759147644),
 ('tremendo', 0.7876365184783936),
 ('ebbrezza', 0.7850828170776367),
 ('ribrezzo', 0.7829298377037048)]

In [41]:
w2v6.save(os.path.join('trained_models/Word2Vec10', '10w2v6.model'))

### Zeitraum 7: 1901-1925

In [42]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [43]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 18.9 s


In [44]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 13s


In [45]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('impeto', 0.8587533831596375),
 ('inquietudine', 0.8497499823570251),
 ('indignazione', 0.8454335927963257),
 ('furore', 0.844071626663208),
 ('commozione', 0.8410149216651917),
 ('orrore', 0.8408750295639038),
 ('indicibile', 0.8404538035392761),
 ('angoscia', 0.8392099738121033),
 ('ebbrezza', 0.8325240612030029),
 ('subitaneo', 0.8296780586242676)]

In [46]:
w2v7.save(os.path.join('trained_models/Word2Vec10', '10w2v7.model'))

### Zeitraum 8: 1926-1950

In [47]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [48]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 17.1 s


In [49]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 16s


In [50]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.8957478404045105),
 ('ribrezzo', 0.8525710105895996),
 ('raccapriccio', 0.8525347709655762),
 ('subitaneo', 0.8476174473762512),
 ('impeto', 0.8441795110702515),
 ('furore', 0.8439134359359741),
 ('stupore', 0.8436235189437866),
 ('sussulto', 0.8426586389541626),
 ('disperazione', 0.8401604294776917),
 ('sgomento', 0.839021623134613)]

In [51]:
w2v8.save(os.path.join('trained_models/Word2Vec10', '10w2v8.model'))

### Zeitraum 9: 1951-1985

In [52]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [53]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 18.4 s


In [54]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 19s


In [55]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('sconvolgere', 0.8206798434257507),
 ('panico', 0.8001352548599243),
 ('orrore', 0.787828803062439),
 ('angoscia', 0.7751887440681458),
 ('smarrimento', 0.7670857310295105),
 ('tremendo', 0.7638232707977295),
 ('annientare', 0.7610354423522949),
 ('tumulto', 0.7601290941238403),
 ('oscurare', 0.7568005323410034),
 ('sgomento', 0.7531386613845825)]

In [56]:
w2v9.save(os.path.join('trained_models/Word2Vec10', '10w2v9.model'))

### Zeitraum 10: 1986-2000

In [57]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [58]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 35 s


In [59]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 17s


In [60]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.8477425575256348),
 ('rabbia', 0.8423449993133545),
 ('lutto', 0.841077446937561),
 ('cieco', 0.8355154395103455),
 ('tremendo', 0.834452748298645),
 ('preda', 0.829306960105896),
 ('disperazione', 0.8261551260948181),
 ('dittatura', 0.8155891299247742),
 ('terribile', 0.8141192197799683),
 ('scoppio', 0.8120735883712769)]

In [61]:
w2v10.save(os.path.join('trained_models/Word2Vec10', '10w2v10.model'))

### Zeitraum 11: 2001-2021

In [62]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [63]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 13 s


In [64]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 26s


In [65]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.9040887355804443),
 ('preda', 0.9005751013755798),
 ('seminare', 0.8989688158035278),
 ('disperazione', 0.8989484310150146),
 ('angoscia', 0.8970781564712524),
 ('crudele', 0.8957701325416565),
 ('incutere', 0.8878142237663269),
 ('tristezza', 0.8830910921096802),
 ('disprezzo', 0.880854070186615),
 ('orrore', 0.880737841129303)]

In [66]:
w2v11.save(os.path.join('trained_models/Word2Vec10', '10w2v11.model'))